In [54]:
import os
import json
import pandas as pd
import traceback

In [55]:
from langchain.chat_models import ChatOpenAI

In [56]:
from dotenv import load_dotenv

load_dotenv()

True

In [57]:
openai_key = os.getenv("OPENAI_API_KEY")

In [58]:
llm = ChatOpenAI(api_key = openai_key, model_name="gpt-3.5-turbo", temperature=0.7)

In [59]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [60]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [61]:
TEMPLATE = """

Material: {material}

You are a knowledgable professor, who is an expert at creating multiple choice questions. Given the above material, it is your job to \
create a multiple choice quiz of {number} multiple choice questions for students of {subject}. Set the difficulty of the exam to be {difficulty}.
Make sure the questions are not repeated and check to make sure that students can get the answer to each of your questions using the above text, and \
the above text alone.

Make sure to format your response like RESPONSE_JSON below, and use it as a guide for your output.

Ensure there are {number} Multiple Choice Questions.

### Response JSON below:
{response_json}

"""

In [62]:
quiz_gen_prompt = PromptTemplate(
    input_variables=["material", "number", "subject", "difficulty", "response_json"],
    template=TEMPLATE
)

In [63]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_gen_prompt, output_key="quiz", verbose=True)

In [64]:
TEMPLATE_EVAL = """
You are an expert in English grammar and writing. You've been tasked with evaluating a Multiple Choice Quiz for {subject} students. \
Your analysis should include a concise assessment of its complexity, using a maximum of 50 words. \
If the quiz doesn't match the students' cognitive and analytical abilities, modify questions accordingly, adjusting the tone to suit their level.

Quiz_MCQs:
{quiz}

Seek evaluation from an expert English writer for the quiz above:
"""


In [65]:
quiz_eval_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE_EVAL
)

In [66]:
review_chain = LLMChain(llm=llm, prompt=quiz_eval_prompt, output_key="review", verbose=True)

In [67]:
generate_evaluation_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["material", "number", "subject", "difficulty", "response_json"], 
                                            output_variables = ["quiz", "review"], verbose=True)

In [68]:
file_path = r'C:\Users\03123\OM\NLP\mcq_gen\data.txt'

In [69]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [70]:
print(TEXT)

Hinduism as it is commonly known can be subdivided into a number of major currents. Of the historical division into six darsanas (philosophies), two schools, Vedanta and Yoga, are currently the most prominent.[62] The six āstika schools of Hindu philosophy, which recognise the authority of the Vedas are: Sānkhya, Yoga, Nyāya, Vaisheshika, Mimāmsā, and Vedānta.[4][5]

Classified by primary deity or deities, four major Hinduism modern currents are Shaivism (Shiva), Vaishnavism (Vishnu), Shaktism (Devi) and Smartism (five deities treated as equals).[63][64][65][66] Hinduism also accepts numerous divine beings, with many Hindus considering the deities to be aspects or manifestations of a single impersonal absolute or ultimate reality or Supreme God, while some Hindus maintain that a specific deity represents the supreme and various deities are lower manifestations of this supreme.[67] Other notable characteristics include a belief in the existence of ātman (self), reincarnation of one's āt

In [71]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [72]:
NUMBER = 5
SUBJECT = 'religion'
TONE = 'easy'


In [73]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluation_chain(
        {
            "material": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "difficulty": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Material: Hinduism as it is commonly known can be subdivided into a number of major currents. Of the historical division into six darsanas (philosophies), two schools, Vedanta and Yoga, are currently the most prominent.[62] The six āstika schools of Hindu philosophy, which recognise the authority of the Vedas are: Sānkhya, Yoga, Nyāya, Vaisheshika, Mimāmsā, and Vedānta.[4][5]

Classified by primary deity or deities, four major Hinduism modern currents are Shaivism (Shiva), Vaishnavism (Vishnu), Shaktism (Devi) and Smartism (five deities treated as equals).[63][64][65][66] Hinduism also accepts numerous divine beings, with many Hindus considering the deities to be aspects or manifestations of a single impersonal absolute or ultimate reality or Supreme God, while some Hindus maintain that a specific deity represents the supreme and various deities are lower manifestations of this supre

In [74]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")


Total Tokens:2617
Prompt Tokens:2061
Completion Tokens:556
Total Cost:0.004203500000000001


In [76]:
response

{'material': 'Hinduism as it is commonly known can be subdivided into a number of major currents. Of the historical division into six darsanas (philosophies), two schools, Vedanta and Yoga, are currently the most prominent.[62] The six āstika schools of Hindu philosophy, which recognise the authority of the Vedas are: Sānkhya, Yoga, Nyāya, Vaisheshika, Mimāmsā, and Vedānta.[4][5]\n\nClassified by primary deity or deities, four major Hinduism modern currents are Shaivism (Shiva), Vaishnavism (Vishnu), Shaktism (Devi) and Smartism (five deities treated as equals).[63][64][65][66] Hinduism also accepts numerous divine beings, with many Hindus considering the deities to be aspects or manifestations of a single impersonal absolute or ultimate reality or Supreme God, while some Hindus maintain that a specific deity represents the supreme and various deities are lower manifestations of this supreme.[67] Other notable characteristics include a belief in the existence of ātman (self), reincarna

In [77]:
quiz = response.get("quiz")

In [83]:
quiz_json = json.loads(quiz)

In [79]:
review = response.get("review")

In [80]:
review

'The Multiple Choice Quiz for religion students is complex and may not match their cognitive and analytical abilities. Questions could be modified to simplify language and structure, ensuring better comprehension for students. Tone adjustment is recommended to suit their level of understanding.'

In [84]:
quiz_table_data = []
for key, value in quiz_json.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [85]:
quiz_table_data

[{'MCQ': 'Which of the following is not one of the six major kinds of Hinduism according to June McDaniel?',
  'Choices': 'a: Folk Hinduism | b: Vedic Hinduism | c: Buddhism | d: Vedantic Hinduism',
  'Correct': 'c'},
 {'MCQ': 'What are the four major currents of Hinduism based on primary deities?',
  'Choices': 'a: Shaivism, Vaishnavism, Shaktism, Smartism | b: Sānkhya, Yoga, Nyāya, Vaisheshika | c: Ramaism, Brahmo Samaj, Theosophical Society, ISKCON | d: Vedanta, Yoga, Nyāya, Vaisheshika',
  'Correct': 'a'},
 {'MCQ': 'According to Michaels, what are the three Hindu religions?',
  'Choices': 'a: Brahmanic-Sanskritic Hinduism, Jainism, Sikhism | b: Brahmanic-Sanskritic Hinduism, folk religions and tribal religions, founded religions | c: Vaishnavism, Shaivism, Shaktism | d: Vedanta, Yoga, Mimāmsā',
  'Correct': 'b'},
 {'MCQ': 'What is the classification of Hindu religiosity according to Michaels?',
  'Choices': 'a: karma-marga, jnana-marga, bhakti-marga, heroism | b: Buddhism, Jainism,

In [87]:
quiz_df = pd.DataFrame(quiz_table_data)

In [88]:
quiz_df.to_csv("hinduism.csv", index=False)